In [1]:
import cherrypy
import json
import redis

In [2]:
REDIS_HOST = 'redis-19466.c293.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 19466
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'iIQjppRdJ7f3kKU4Ng5IDkgv6TvaPnrN'
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


In [3]:
# Endpoint /devices
class Devices(object):
    exposed = True
    def GET(self, *path, **query):
        devices_list = list({key.decode().split(':')[0] for key in redis_client.keys()})
        devices_json = json.dumps({'mac_addresses': devices_list})
        return devices_json
class Device(object):
    exposed = True
    def GET(self, *path, **query):
        # query must contain from to
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Not Found: missing MAC address.')
        #check if present
        mac_address_ = path[0]
        devices_list = list({key.decode().split(':')[0] for key in redis_client.keys()})
        if mac_address_ not in devices_list:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')

        from_ = query.get('from', None)
        to_ = query.get('to', None)
        if from_ is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')
        if to_ is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')
        from_int = int(from_)
        to_int = int(to_)
        battery_ts = redis_client.ts().range(key=mac_address_+':battery', from_time=from_int, to_time=to_int)
        power_ts = redis_client.ts().range(key=mac_address_+':power', from_time=from_int, to_time=to_int)
        timestamp_list = [time for time,value in battery_ts]
        battery_list = [value for time,value in battery_ts]
        power_list = [value for time,value in power_ts]
        return json.dumps({
            'mac_address': mac_address_,
            'timestamps': timestamp_list,
            'battery_levels': battery_list,
            'power_plugged': power_list,
        })
    def DELETE(self,*path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')
        mac_address_ = path[0]
        devices_list = list({key.decode().split(':')[0] for key in redis_client.keys()})
        if mac_address_ not in devices_list:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')
        redis_client.delete(mac_address_+':battery')
        redis_client.delete(mac_address_+':power')
        print('Deleted')

In [4]:
conf = {
    '/' : {'request.dispatch' : cherrypy.dispatch.MethodDispatcher() } # configuration of the path to address
}
cherrypy.tree.mount(Devices(), '/devices', conf) # one for each object + passing the configuration
cherrypy.tree.mount(Device(), '/device', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socketport': 8080})
cherrypy.engine.start()
cherrypy.engine.block()

[24/Jan/2023:19:17:31] ENGINE Bus STARTING
[24/Jan/2023:19:17:31] ENGINE Started monitor thread 'Autoreloader'.
[24/Jan/2023:19:17:31] ENGINE Serving on http://0.0.0.0:8080
[24/Jan/2023:19:17:31] ENGINE Bus STARTED
[24/Jan/2023:19:28:21] ENGINE Keyboard Interrupt: shutting down bus
[24/Jan/2023:19:28:21] ENGINE Bus STOPPING
[24/Jan/2023:19:28:21] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 8080)) shut down
[24/Jan/2023:19:28:21] ENGINE Stopped thread 'Autoreloader'.
[24/Jan/2023:19:28:21] ENGINE Bus STOPPED
[24/Jan/2023:19:28:21] ENGINE Bus EXITING
[24/Jan/2023:19:28:21] ENGINE Bus EXITED
[24/Jan/2023:19:28:21] ENGINE Waiting for child threads to terminate...
